In [5]:
"""
Reasoning Capability Detection & Caching
Unified probe with memory + persistent disk cache with TTL expiration
"""
import json
from datetime import datetime, timedelta
from pathlib import Path
from typing import Dict, Optional, Tuple

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama


class ReasoningCapabilityProbe:
    """
    Probes LLM for reasoning mode support with intelligent caching.
    Maintains an in-memory cache and optional persistent cache file with TTL.

    ✔ UNIFIED: Single source of truth for reasoning capability detection
    ✔ SMART CACHE: Memory + disk persistence with 24h TTL expiration
    ✔ FAST PATH: Pattern matching for known reasoning models (no probe)
    ✔ PRODUCTION: Comprehensive error handling, diagnostics, management
    """

    CACHE_DIR = Path(".cache")
    CACHE_FILE = CACHE_DIR / "reasoning_capabilities.json"
    CACHE_TTL_HOURS = 480  # Cache validity period

    def __init__(self, show_progress: bool = True, use_persistent_cache: bool = True):
        """
        Initialize probe with caching options.

        Args:
            show_progress: Log probe activities
            use_persistent_cache: Save/load cache to disk
        """
        self.show_progress = show_progress
        self.use_persistent_cache = use_persistent_cache
        self.memory_cache: Dict[str, Dict] = {}

        if self.use_persistent_cache:
            self._load_persistent_cache()

    def _load_persistent_cache(self) -> None:
        """Load cached capabilities from disk if available."""
        if not self.CACHE_FILE.exists():
            return

        try:
            with open(self.CACHE_FILE, "r") as f:
                data = json.load(f)
                self.memory_cache = data.get("capabilities", {})

            if self.show_progress:
                print(f"✓ Loaded capability cache: {len(self.memory_cache)} models")
                print(f"  Cache file: {self.CACHE_FILE}")

        except Exception as e:
            print(f"Failed to load persistent cache: {e}")
            self.memory_cache = {}

    def _save_persistent_cache(self) -> None:
        """Save current cache to disk."""
        if not self.use_persistent_cache:
            return

        try:
            self.CACHE_DIR.mkdir(exist_ok=True)

            with open(self.CACHE_FILE, "w") as f:
                json.dump(
                    {
                        "timestamp": datetime.now().isoformat(),
                        "ttl_hours": self.CACHE_TTL_HOURS,
                        "capabilities": self.memory_cache,
                    },
                    f,
                    indent=2,
                )

            if self.show_progress:
                print(f"💾 Saved capability cache: {self.CACHE_FILE}")

        except Exception as e:
            print(f"Failed to save persistent cache: {e}")

    def _is_cache_valid(self, cache_entry: Dict) -> bool:
        """Check if a cache entry is still valid (not expired)."""
        if "timestamp" not in cache_entry:
            return False

        try:
            cached_time = datetime.fromisoformat(cache_entry["timestamp"])
            expiry_time = cached_time + timedelta(hours=self.CACHE_TTL_HOURS)
            is_valid = datetime.now() < expiry_time

            if not is_valid and self.show_progress:
                print(f"  Cache entry expired at {expiry_time.isoformat()}")

            return is_valid

        except Exception as e:
            print(f"  Invalid cache timestamp: {e}")
            return False

    def check_reasoning_support(
            self,
            model: str,
            temperature: float = 1.0,
            context_window: int = 4096,
    ) -> Tuple[bool, str]:
        """
        Check if model supports reasoning mode with caching.

        Args:
            model: Model name/identifier
            temperature: LLM temperature setting
            context_window: LLM context window size

        Returns:
            (supports_reasoning: bool, reason: str)
        """
        model_key = f"{model}#{temperature}#{context_window}"

        # ✔ CHECK MEMORY CACHE FIRST
        if model_key in self.memory_cache:
            entry = self.memory_cache[model_key]
            if self._is_cache_valid(entry):
                if self.show_progress:
                    print(f"📦 Reasoning capability [CACHED]: {model}")
                    print(f"  Result: {entry['supports_reasoning']}")
                    print(f"  Reason: {entry['reason']}")

                return entry["supports_reasoning"], entry["reason"]

        # ✔ NOT CACHED OR EXPIRED - PROBE THE MODEL
        if self.show_progress:
            print(f"🔍︎ Probing reasoning capability: {model}")
            print(f"  Temperature: {temperature}")
            print(f"  Context window: {context_window}")

        supports, reason = self._probe_model(model, temperature, context_window)

        # ✔ CACHE THE RESULT
        self.memory_cache[model_key] = {
            "model": model,
            "temperature": temperature,
            "context_window": context_window,
            "supports_reasoning": supports,
            "reason": reason,
            "timestamp": datetime.now().isoformat(),
        }

        self._save_persistent_cache()

        if self.show_progress:
            status = "✓ SUPPORTED" if supports else "✗ NOT SUPPORTED"
            print(f"  {status}: {reason}")

        return supports, reason

    def _probe_model(
            self, model: str, temperature: float, context_window: int
    ) -> Tuple[bool, str]:
        """
        Attempt to use reasoning mode and detect support.

        Returns:
            (success: bool, reason: str)
        """
        # ✔ PATTERN MATCHING FOR KNOWN REASONING MODELS (FAST PATH)
        known_reasoning_models = [
            "deepseek-r1",
            "qwen-qwq",
            "qwen-r",
            "o1",
            "reasoning",
            "r1-preview",
        ]

        for pattern in known_reasoning_models:
            if pattern in model.lower():
                return True, f"Model matches known reasoning pattern: '{pattern}'"

        # ✔ ATTEMPT ACTUAL PROBE
        try:
            llm = ChatOllama(
                model=model,
                temperature=temperature,
                num_ctx=context_window,
                num_predict=512,  # Small token limit for probe
                reasoning=True,
            )

            messages = [
                SystemMessage(
                    content=(
                        "You are a reasoning model. "
                        "Respond with a single JSON object: "
                        '{"reasoning_capable": true}'
                    )
                ),
                HumanMessage(content="Confirm reasoning mode support."),
            ]

            response = llm.invoke(messages)

            # Check if response indicates reasoning mode
            if (
                    "reasoning_capable" in response.content.lower()
                    or "true" in response.content.lower()
            ):
                return True, "Reasoning mode test passed"

            return False, "Model responded but reasoning mode unconfirmed"

        except TypeError as e:
            if "reasoning" in str(e).lower():
                return (
                    False,
                    f"Model does not support reasoning parameter: {str(e)[:80]}",
                )
            return False, f"Parameter error (reasoning unsupported): {str(e)[:80]}"

        except Exception as e:
            error_type = type(e).__name__
            error_msg = str(e)[:80]

            if any(
                    term in error_msg.lower()
                    for term in ["unknown parameter", "unexpected", "reasoning"]
            ):
                return (
                    False,
                    f"{error_type}: Model does not support reasoning",
                )

            return False, f"{error_type}: Probe failed - {error_msg}"

    def clear_cache(self, model: Optional[str] = None) -> None:
        """
        Clear cache entries.

        Args:
            model: Clear specific model, or None to clear all
        """
        if model is None:
            self.memory_cache.clear()
            if self.CACHE_FILE.exists():
                self.CACHE_FILE.unlink()
            if self.show_progress:
                print("🗑️  Cleared all capability cache")
        else:
            keys_to_remove = [k for k in self.memory_cache.keys() if k.startswith(model)]
            for key in keys_to_remove:
                del self.memory_cache[key]
            self._save_persistent_cache()
            if self.show_progress:
                print(f"🗑️  Cleared cache for: {model} ({len(keys_to_remove)} entries)")

    def get_cache_stats(self) -> Dict:
        """Return cache statistics."""
        total_entries = len(self.memory_cache)
        valid_entries = sum(
            1
            for entry in self.memory_cache.values()
            if self._is_cache_valid(entry)
        )
        supported = sum(
            1
            for entry in self.memory_cache.values()
            if entry.get("supports_reasoning", False)
        )

        return {
            "total_entries": total_entries,
            "valid_entries": valid_entries,
            "supported_models": supported,
            "cache_file": str(self.CACHE_FILE),
            "ttl_hours": self.CACHE_TTL_HOURS,
        }

In [6]:
model = "gpt-oss:20b"
temperature = 0
context_window = 512

probe = ReasoningCapabilityProbe(show_progress=True)
reasoning_supported, reasoning_reason = probe.check_reasoning_support(
    model=model,
    temperature=temperature,
    context_window=context_window,
)


print("🔧 Reasoning Engine Initialized")
print(f"  Model: {model}")
print(f"  Temperature: {temperature}")
print(f"  Context window: {context_window}")
print(f"  Reasoning supported (probed): {reasoning_supported}")
print(f"  Probe reason: {reasoning_reason}")


✓ Loaded capability cache: 1 models
  Cache file: .cache\reasoning_capabilities.json
📦 Reasoning capability [CACHED]: gpt-oss:20b
  Result: True
  Reason: Reasoning mode test passed
🔧 Reasoning Engine Initialized
  Model: gpt-oss:20b
  Temperature: 0
  Context window: 512
  Reasoning supported (probed): True
  Probe reason: Reasoning mode test passed
